In [24]:
#국가별 GDP대비 토탈 메달수
import pandas as pd
import plotly.graph_objects as go
import os
import webbrowser

# 1. CSV 파일 불러오기
df = pd.read_csv("1_medals_gdp.csv")

# 2. 필터링 대상 국가 및 연도
target_iso3 = ["KOR", "USA", "CHN", "JPN", "GBR", "FRA", "AUS", "ITA"]
olympic_years = list(range(1984, 2025, 4))
official_2024_medals = {
    "USA": 126, "CHN": 91, "JPN": 45,
    "GBR": 64, "FRA": 28, "AUS": 50, "KOR": 32, "ITA": 40
}

# 3. 데이터 전처리
df = df[df["iso3"].isin(target_iso3)].copy()
completed_rows = []
for iso in target_iso3:
    country_name = df[df["iso3"] == iso]["country_name"].iloc[0]
    gdp_2020 = df[(df["iso3"] == iso) & (df["Year"] == 2020)]["GDP"]
    pop_2020 = df[(df["iso3"] == iso) & (df["Year"] == 2020)]["Population"]
    gdp_2020 = gdp_2020.iloc[0] if not gdp_2020.empty else 1e9
    pop_2020 = pop_2020.iloc[0] if not pop_2020.empty else 1e6
    total_2024 = official_2024_medals.get(iso, 0)

    exists_2024 = ((df["iso3"] == iso) & (df["Year"] == 2024)).any()
    if exists_2024:
        df.loc[(df["iso3"] == iso) & (df["Year"] == 2024), ["GDP", "Population", "Total"]] = [gdp_2020, pop_2020, total_2024]
    else:
        completed_rows.append({
            "iso3": iso,
            "country_name": country_name,
            "Year": 2024,
            "GDP": gdp_2020,
            "Population": pop_2020,
            "Total": total_2024
        })

df = pd.concat([df, pd.DataFrame(completed_rows)], ignore_index=True)
df = df.dropna(subset=["GDP", "Population", "Total"])
df["GDP"] = df["GDP"] / 1e6  # 백만 달러 단위로 변환

# 4. 차트 범위 및 버블 크기 계산
max_gdp = df["GDP"].max()
max_pop = df["Population"].max()
sizeref = 2. * max_pop / (240 ** 2)
center_x = (-5 + max_gdp * 1.1) / 2
top_y = 190

# 5. 초기 프레임 (1984년)
init_data = df[df["Year"] == 1984]
avg_gdp_init = init_data["GDP"].mean()
avg_medals_init = init_data["Total"].mean()
initial_shapes = [
    dict(type="line", x0=avg_gdp_init, x1=avg_gdp_init, y0=0, y1=200,
         line=dict(color="gray", width=2, dash="dot")),
    dict(type="line", x0=-5, x1=max_gdp, y0=avg_medals_init, y1=avg_medals_init,
         line=dict(color="gray", width=2, dash="dot"))
]
init_annotation_top = [
    dict(
        x=center_x,
        y=top_y,
        xref='x',
        yref='y',
        text=f"<b>1984 Avg GDP: {avg_gdp_init:,.0f}M / Avg Medals: {avg_medals_init:.1f}</b>",
        showarrow=False,
        font=dict(size=16, color="black"),
        xanchor="center",
        yanchor="bottom"
    )
]

# 6. 초기 데이터 trace
init_data_traces_with_hover = []
for iso in target_iso3:
    row = init_data[init_data["iso3"] == iso]
    if not row.empty:
        hovertext = (
            f"Country: {row['country_name'].values[0]}<br>"
            f"Year: 1984<br>"
            f"Total Medals: {row['Total'].values[0]}<br>"
            f"GDP: {row['GDP'].values[0]:,.0f}M USD<br>"
            f"Population: {row['Population'].values[0]:,.0f}"
        )
        init_data_traces_with_hover.append(go.Scatter(
            x=[row["GDP"].values[0]],
            y=[row["Total"].values[0]],
            mode="markers+text",
            text=[iso],
            textposition="middle center",
            textfont=dict(size=20, color="black"),
            marker=dict(
                size=[row["Population"].values[0]],
                sizemode="area",
                sizeref=sizeref,
                sizemin=6,
                line=dict(width=1, color="DarkSlateGrey")
            ),
            name=iso,
            hovertext=hovertext,
            hoverinfo="text",
            showlegend=True
        ))

# 7. 연도별 프레임 생성
frames_with_hover = []
for year in olympic_years:
    frame_data = df[df["Year"] == year]
    data = []

    avg_gdp = frame_data["GDP"].mean()
    avg_medals = frame_data["Total"].mean()

    shapes = [
        dict(type="line", x0=avg_gdp, x1=avg_gdp, y0=0, y1=200,
             line=dict(color="gray", width=2, dash="dot")),
        dict(type="line", x0=-5, x1=max_gdp, y0=avg_medals, y1=avg_medals,
             line=dict(color="gray", width=2, dash="dot"))
    ]

    annotations = [
        dict(
            x=center_x,
            y=top_y,
            xref='x',
            yref='y',
            text=f"<b>{year} Avg GDP: {avg_gdp:,.0f}M / Avg Medals: {avg_medals:.1f}</b>",
            showarrow=False,
            font=dict(size=16, color="black"),
            xanchor="center",
            yanchor="bottom"
        )
    ]

    for iso in target_iso3:
        row = frame_data[frame_data["iso3"] == iso]
        if not row.empty:
            hovertext = (
                f"Country: {row['country_name'].values[0]}<br>"
                f"Year: {year}<br>"
                f"Total Medals: {row['Total'].values[0]}<br>"
                f"GDP: {row['GDP'].values[0]:,.0f}M USD<br>"
                f"Population: {row['Population'].values[0]:,.0f}"
            )
            data.append(go.Scatter(
                x=[row["GDP"].values[0]],
                y=[row["Total"].values[0]],
                mode="markers+text",
                text=[row["iso3"].values[0]],
                textposition="middle center",
                textfont=dict(size=20, color="black"),
                marker=dict(
                    size=[row["Population"].values[0]],
                    sizemode="area",
                    sizeref=sizeref,
                    sizemin=6,
                    line=dict(width=1, color="DarkSlateGrey")
                ),
                name=iso,
                hovertext=hovertext,
                hoverinfo="text",
                showlegend=True
            ))

    frames_with_hover.append(go.Frame(data=data, name=str(year), layout=go.Layout(shapes=shapes, annotations=annotations)))

# 8. Figure 생성
fig = go.Figure(
    data=init_data_traces_with_hover,
    layout=go.Layout(
        title="Olympic Medals vs GDP (1984–2024, Bubble = Population, Hover Info Enabled)",
        xaxis=dict(title="GDP (Million USD)", range=[-2000000, max_gdp * 1.1]),
        yaxis=dict(title="Total Olympic Medals", range=[0, 200]),
        shapes=initial_shapes,
        annotations=init_annotation_top,
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play", method="animate",
                     args=[None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]),
                dict(label="Pause", method="animate",
                     args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}])
            ]
        )],
        sliders=[dict(
            active=0,
            steps=[
                dict(label=str(year), method="animate",
                     args=[[str(year)], {"frame": {"duration": 600, "redraw": True}, "mode": "immediate"}])
                for year in olympic_years
            ]
        )],
        height=850,
        width=1300,
        showlegend=True
    ),
    frames=frames_with_hover
)

# 9. 저장 및 자동 열기
output_path = "olympic_gdp_medals_hoverinfo_final.html"
fig.write_html(output_path)
webbrowser.open("file://" + os.path.abspath(output_path))
print("국가별 GDP 대비 메달수 HTML 완료")


국가별 GDP 대비 메달수 HTML 완료


In [25]:
#국가별 1인당 GDP별 메달 수
import pandas as pd
import plotly.graph_objects as go
import os
import webbrowser

# 1. CSV 파일 불러오기
df = pd.read_csv("1_medals_gdp_percapita.csv")

# 2. 필터링 대상 국가 및 연도
target_iso3 = ["KOR", "USA", "CHN", "JPN", "GBR", "FRA", "AUS", "ITA"]
olympic_years = list(range(1984, 2025, 4))
official_2024_medals = {
    "USA": 126, "CHN": 91, "JPN": 45,
    "GBR": 64, "FRA": 28, "AUS": 50, "KOR": 32, "ITA": 40
}

# 3. 데이터 전처리
df = df[df["iso3"].isin(target_iso3)].copy()
completed_rows = []
for iso in target_iso3:
    country_name = df[df["iso3"] == iso]["country_name"].iloc[0]
    gdp_2020 = df[(df["iso3"] == iso) & (df["Year"] == 2020)]["GDP"]
    pop_2020 = df[(df["iso3"] == iso) & (df["Year"] == 2020)]["Population"]
    gdp_2020 = gdp_2020.iloc[0] if not gdp_2020.empty else 1e9
    pop_2020 = pop_2020.iloc[0] if not pop_2020.empty else 1e6
    total_2024 = official_2024_medals.get(iso, 0)

    gdp_per_capita = gdp_2020 / pop_2020

    exists_2024 = ((df["iso3"] == iso) & (df["Year"] == 2024)).any()
    if exists_2024:
        df.loc[(df["iso3"] == iso) & (df["Year"] == 2024), ["GDP", "Population", "Total", "GDP_per_capita"]] = [gdp_2020, pop_2020, total_2024, gdp_per_capita]
    else:
        completed_rows.append({
            "iso3": iso,
            "country_name": country_name,
            "Year": 2024,
            "GDP": gdp_2020,
            "Population": pop_2020,
            "Total": total_2024,
            "GDP_per_capita": gdp_per_capita
        })

df = pd.concat([df, pd.DataFrame(completed_rows)], ignore_index=True)
df = df.dropna(subset=["GDP_per_capita", "Population", "Total"])

# 4. 차트 범위 및 버블 크기 계산
max_gdp_pc = df["GDP_per_capita"].max()
max_pop = df["Population"].max()
sizeref = 2. * max_pop / (240 ** 2)
center_x = (-5 + max_gdp_pc * 1.1) / 2
top_y = 190

# 5. 초기 프레임 (1984년)
init_data = df[df["Year"] == 1984]
avg_gdp_pc_init = init_data["GDP_per_capita"].mean()
avg_medals_init = init_data["Total"].mean()
initial_shapes = [
    dict(type="line", x0=avg_gdp_pc_init, x1=avg_gdp_pc_init, y0=0, y1=200,
         line=dict(color="gray", width=2, dash="dot")),
    dict(type="line", x0=-5, x1=max_gdp_pc, y0=avg_medals_init, y1=avg_medals_init,
         line=dict(color="gray", width=2, dash="dot"))
]
init_annotation_top = [
    dict(
        x=center_x,
        y=top_y,
        xref='x',
        yref='y',
        text=f"<b>1984 Avg GDP per capita: {avg_gdp_pc_init:,.0f} / Avg Medals: {avg_medals_init:.1f}</b>",
        showarrow=False,
        font=dict(size=16, color="black"),
        xanchor="center",
        yanchor="bottom"
    )
]

# 6. 초기 trace
init_data_traces_with_hover = []
for iso in target_iso3:
    row = init_data[init_data["iso3"] == iso]
    if not row.empty:
        hovertext = (
            f"Country: {row['country_name'].values[0]}<br>"
            f"Year: 1984<br>"
            f"Total Medals: {row['Total'].values[0]}<br>"
            f"GDP per capita: {row['GDP_per_capita'].values[0]:,.0f} USD<br>"
            f"Population: {row['Population'].values[0]:,.0f}"
        )
        init_data_traces_with_hover.append(go.Scatter(
            x=[row["GDP_per_capita"].values[0]],
            y=[row["Total"].values[0]],
            mode="markers+text",
            text=[iso],
            textposition="middle center",
            textfont=dict(size=20, color="black"),
            marker=dict(
                size=[row["Population"].values[0]],
                sizemode="area",
                sizeref=sizeref,
                sizemin=6,
                line=dict(width=1, color="DarkSlateGrey")
            ),
            name=iso,
            hovertext=hovertext,
            hoverinfo="text",
            showlegend=True
        ))

# 7. 연도별 프레임 생성
frames_with_hover = []
for year in olympic_years:
    frame_data = df[df["Year"] == year]
    data = []

    avg_gdp_pc = frame_data["GDP_per_capita"].mean()
    avg_medals = frame_data["Total"].mean()

    shapes = [
        dict(type="line", x0=avg_gdp_pc, x1=avg_gdp_pc, y0=0, y1=200,
             line=dict(color="gray", width=2, dash="dot")),
        dict(type="line", x0=-5, x1=max_gdp_pc, y0=avg_medals, y1=avg_medals,
             line=dict(color="gray", width=2, dash="dot"))
    ]

    annotations = [
        dict(
            x=center_x,
            y=top_y,
            xref='x',
            yref='y',
            text=f"<b>{year} Avg GDP per capita: {avg_gdp_pc:,.0f} / Avg Medals: {avg_medals:.1f}</b>",
            showarrow=False,
            font=dict(size=16, color="black"),
            xanchor="center",
            yanchor="bottom"
        )
    ]

    for iso in target_iso3:
        row = frame_data[frame_data["iso3"] == iso]
        if not row.empty:
            hovertext = (
                f"Country: {row['country_name'].values[0]}<br>"
                f"Year: {year}<br>"
                f"Total Medals: {row['Total'].values[0]}<br>"
                f"GDP per capita: {row['GDP_per_capita'].values[0]:,.0f} USD<br>"
                f"Population: {row['Population'].values[0]:,.0f}"
            )
            data.append(go.Scatter(
                x=[row["GDP_per_capita"].values[0]],
                y=[row["Total"].values[0]],
                mode="markers+text",
                text=[row["iso3"].values[0]],
                textposition="middle center",
                textfont=dict(size=20, color="black"),
                marker=dict(
                    size=[row["Population"].values[0]],
                    sizemode="area",
                    sizeref=sizeref,
                    sizemin=6,
                    line=dict(width=1, color="DarkSlateGrey")
                ),
                name=iso,
                hovertext=hovertext,
                hoverinfo="text",
                showlegend=True
            ))

    frames_with_hover.append(go.Frame(data=data, name=str(year), layout=go.Layout(shapes=shapes, annotations=annotations)))

# 8. Figure 생성
fig = go.Figure(
    data=init_data_traces_with_hover,
    layout=go.Layout(
        title="Olympic Medals vs GDP per Capita (1984–2024, Bubble = Population)",
        xaxis=dict(title="GDP per Capita (USD)", range=[-5000, max_gdp_pc * 1.1]),
        yaxis=dict(title="Total Olympic Medals", range=[0, 200]),
        shapes=initial_shapes,
        annotations=init_annotation_top,
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play", method="animate",
                     args=[None, {"frame": {"duration": 1000, "redraw": True}, "fromcurrent": True}]),
                dict(label="Pause", method="animate",
                     args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}])
            ]
        )],
        sliders=[dict(
            active=0,
            steps=[dict(label=str(year), method="animate",
                        args=[[str(year)], {"frame": {"duration": 600, "redraw": True}, "mode": "immediate"}])
                   for year in olympic_years]
        )],
        height=850,
        width=1300,
        showlegend=True
    ),
    frames=frames_with_hover
)

# 9. 저장 및 자동 열기
output_path = "olympic_gdp_percapita_medals.html"
fig.write_html(output_path)
webbrowser.open("file://" + os.path.abspath(output_path))
print("국가별 1인 GDP대비 메달수 HTML 완료")


국가별 1인 GDP대비 메달수 HTML 완료
